In [1]:
from flask import Flask, request, jsonify
import urllib.parse
import joblib
import pandas as pandas
import requests
# Load the saved random forest model for texture: wood
# loaded_model = joblib.load('wood_RF_model.joblib')

#collison flag
collisionOnflag = False

app = Flask(__name__)

ESP32_SERVER_URL = 'http://192.168.4.96:80/receive_message'

@app.route('/')
def index():
    return jsonify({'message': 'Welcome to the FLASK server!'})


@app.route('/collisonRegister', methods=['POST'])
def collisonRegister():
    global collisionOnflag
    try:
        collisionStatus = request.json.get('CollisionStatus')  
        textureName = request.json.get('TextureName')
        
        if collisionStatus is None:
            return jsonify({'Status': '400', 'Error': 'No data received'})
        
        elif collisionStatus not in ['ON', 'OFF']:
            return jsonify({'Status': '400', 'Error': 'Invalid CollisionStatus'})
        
        elif collisionStatus == "ON" and collisionOnflag == False:
            if textureName is None or textureName.strip() == '':
                return jsonify({'Status': '400', 'Error': 'Texture name is empty or null'})
            else:
                #start to listen input for the cane velocity
                #for each velocity input from the cane, use the current texture in use to call the prediction model
                #return the predicted DFT321 waveform value
                collisionOnflag = True
                return jsonify({'Status': '200', 'Message': 'CollisionStatus is ON'})
        elif collisionStatus == "OFF" and collisionOnflag == True:  
            #collisionStatus is 'OFF'
            #stop listening to the cane input
            collisionOnflag = False
            return jsonify({'Status': '200', 'Message': 'CollisionStatus is OFF'})
        else:
            return jsonify({'Status': '400', 'Message': 'CollisionStatus is ALREADY ON'})
    
    except Exception as e:
        return jsonify({'Status': '500', 'Error': str(e)} )
    
# @app.route('/predict', methods=['POST'])
# def predict():
#     global collisionOnflag
#     if collisionOnflag == True:
#         try:
#                 data = request.get_json()
#                 # Convert JSON data to a Python dictionary
#                 vector_data = {
#                     'VELO-X': data.get('x'),
#                     'VELO-Y': data.get('y'),
#                     'VELO-Z': data.get('z')
#                 }

#                 x_input_df = pd.DataFrame([vector_data])
#                 # y_pred = loaded_model.predict(x_input_df)

#                 # return jsonify({'Status': '200', 'Message': str(y_pred)})
#         except Exception as e:
#                 return jsonify({'Status': '500', 'Error': str(e)}) 
#     else:
#         return jsonify({'Status': '400', 'Error': 'Collision flag is not set to true'}) 


@app.route('/esp32', methods=['POST'])
def manual_message():
    
    try:
        json_data = request.get_json()

        if json_data:
            # Convert JSON data to x-www-form-urlencoded format
            form_data = urllib.parse.urlencode(json_data)

            # Forward the data to the ESP32 server
            response = requests.post(ESP32_SERVER_URL, data=form_data, headers={'Content-Type': 'application/x-www-form-urlencoded'})
        
            # Check the response from the ESP32 server
            response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code

            return {'status': 'success', 'message': 'Message forwarded to ESP32 server'}
        else:
            return {'status': 'error', 'message': 'Invalid JSON data in the request'}, 400
    except requests.exceptions.RequestException as e:
        # Handle any exceptions related to the HTTP request
        return {'status': 'error', 'message': f'Error forwarding message to ESP32 server: {str(e)}'}, 500
    except Exception as e:
        # Handle any other unexpected exceptions
        return {'status': 'error', 'message': f'Unexpected error: {str(e)}'}, 500



if __name__ == '__main__':
    # app.run(host='0.0.0.0', port=8080)
    app.run(host = '0.0.0.0', port = '8080')



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.4.57:8080
Press CTRL+C to quit
127.0.0.1 - - [15/Jan/2024 19:18:09] "POST /manual_message HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 19:22:23] "POST /manual_message HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 19:22:29] "POST /manual_message HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 19:22:35] "POST /manual_message HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 19:23:04] "POST /manual_message HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 19:23:40] "POST /manual_message HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 19:23:44] "POST /manual_message HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 19:28:12] "POST /manual_message HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 19:28:15] "POST /manual_message HTTP/1.1" 200 -
